<a href="https://colab.research.google.com/github/Sriramdayal/Unsloth-LLM-finetuningv1/blob/main/Qwen_LeetCoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Local Inference on GPU
Model page: https://huggingface.co/black279/Qwen_LeetCoder

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/black279/Qwen_LeetCoder)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [1]:
!nvidia-smi


Thu Jan 29 18:45:36 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 591.74                 Driver Version: 591.74         CUDA Version: 13.1     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   45C    P8              3W /   80W |    1970MiB /   8188MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:

!pip install -U \
  torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 1.4 MB/s eta 0:29:17
     ---------------------------------------- 0.0/2.4 GB 1.9 MB/s eta 0:21:08
     ---------------------------------------- 0.0/2.4 GB 2.5 MB/s eta 0:16:21
     ---------------------------------------- 0.0/2.4 GB 2.6 MB/s eta 0:15:57
     ---------------------------------------- 0.0/2.4 GB 3.3 MB/s eta 0:12:30
     ---------------------------------------- 0.0/2.4 GB 3.5 MB/s eta 0:11:45
     ---------------------------------------- 0.0/2.4 GB 4.0 MB/s eta 0:10:07
     ---------------------------------------- 0.0/2.4 GB 4.1 MB/s eta 0:09:53
     ----------------------------

In [2]:
!pip install -U transformers peft accelerate bitsandbytes


  Using cached peft-0.18.0-py3-none-any.whl.metadata (14 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.49.0-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached peft-0.18.0-py3-none-any.whl (556 kB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)
Using cached bitsandbytes-0.49.0-py3-none-manylinux_2_24_x86_64.whl (59.1 MB)


In [3]:
import torch
import bitsandbytes as bnb
import transformers
import peft

print("CUDA:", torch.cuda.is_available())
print("Torch:", torch.__version__)
print("BNB:", bnb.__version__)
print("Transformers:", transformers.__version__)
print("PEFT:", peft.__version__)


CUDA: True
Torch: 2.5.1+cu121
BNB: 0.49.0
Transformers: 4.57.3
PEFT: 0.18.0


In [4]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

BASE_MODEL = "unsloth/qwen2.5-0.5b-unsloth-bnb-4bit"
LORA_MODEL = "black279/Qwen_LeetCoder"

tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

model = PeftModel.from_pretrained(
    base_model,
    LORA_MODEL
)

model.eval()


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/521M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/141M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
            

In [6]:
prompt = "Solve: Write a function to find the longest common prefix string amongst an array of strings."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))


Solve: Write a function to find the longest common prefix string amongst an array of strings. If there is no common prefix, return an empty string `""`.

"""

class Solution:

    def longestCommonPrefix(self, strs: List[str]) -> str:

### "class Solution:\n  def longestCommonPrefix(self, strs: List[str]) -> str:\n    if not strs:\n      return ''\n\n    prefix = ''\n    for i in range(len(strs[0])):\n      seen = set()\n      for j in range(1, len(strs)):\n        if j == len(strs) or i == len(strs[j]):\n          return prefix\n        if i == len(strs[j]) or strs[j][i] != strs[0][i]:\n          return prefix\n        seen.add(strs[j][i])\n      prefix += strs[j][i]\n\n    return prefix\n"
